In [1]:
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain.chat_models import ChatOpenAI
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from typing import TypedDict, Annotated
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os
from langgraph.checkpoint.memory import InMemorySaver


In [2]:
load_dotenv()

PERPLEXITY_API_KEY = os.getenv("PERPLEXITY_API_KEY")

model = ChatOpenAI(
    model="sonar",                     # base Sonar model
    base_url="https://api.perplexity.ai",
    api_key=PERPLEXITY_API_KEY,
    temperature=0.7
)

In [3]:
class ChatState(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]

In [4]:
checkPoint = InMemorySaver()


In [5]:
def chat_node(state: ChatState):
    messages = state['messages']
    response = model.invoke(messages)
    return {"messages": [response]}


In [6]:
graph = StateGraph(ChatState)

graph.add_node("chat_node", chat_node)
graph.add_edge(START, "chat_node")
graph.add_edge("chat_node", END)

chatbot = graph.compile()

In [7]:
chatbot.invoke({"messages": [HumanMessage(content="Hello! How are you?")]}, checkpoint=checkPoint)

{'messages': [HumanMessage(content='Hello! How are you?', additional_kwargs={}, response_metadata={}, id='28eee62c-7962-47b3-a0b4-e39d03f6ded0'),
  AIMessage(content='I\'m doing well, thank you! How can I assist you today?\n\nIn English, common ways to respond to "Hello! How are you?" include casual replies such as "I\'m good, thanks," "I\'m great," or "I\'m wonderful." People often follow up with a reciprocal question like "And you?" or "How about you?" for a natural conversation flow[1][2][3]. If you\'d like, I can also share various ways to greet someone or respond in different social contexts.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 105, 'prompt_tokens': 6, 'total_tokens': 111, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'search_context_size': 'low', 'cost': {'input_tokens_cost': 0.0, 'output_tokens_cost': 0.0, 'request_cost': 0.005, 'total_cost': 0.005}}, 'model_name': 'sonar', 'system_fingerprint': None,